# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [173]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
import scipy as sp
import re
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LocalAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [105]:
# load data from database
engine = create_engine('sqlite:///MessagesDB.db')
df = pd.read_sql_table('MessagesDB',engine)
X = df.message.values
y = df.drop(columns=df.columns[0:4],axis=1)


### 2. Write a tokenization function to process your text data

In [106]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)

    tokens = [WordNetLemmatizer().lemmatize(w).strip() for w in tokens if w not in stopwords.words("english")]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [107]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=MultinomialNB()))
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.3)
pipeline.fit(X_train , y_train)


c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000221C47A9090>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [109]:
y_pred = pipeline.predict(X_test)

In [142]:
for col in range(y_pred.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred[:,col],labels=np.unique(y_pred[:,col])))
    print(col)


              precision    recall  f1-score   support

           0       0.78      0.10      0.17      1844
           1       0.78      0.99      0.87      5966

   micro avg       0.78      0.78      0.78      7810
   macro avg       0.78      0.54      0.52      7810
weighted avg       0.78      0.78      0.71      7810

0
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      6488
           1       0.87      0.21      0.34      1377

    accuracy                           0.86      7865
   macro avg       0.86      0.60      0.63      7865
weighted avg       0.86      0.86      0.82      7865

1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838

   micro avg       1.00      1.00      1.00      7838
   macro avg       1.00      1.00      1.00      7838
weighted avg       1.00      1.00      1.00      7838

2
              precision    recall  f1-score   support

           0

### 6. Improve your model
Use grid search to find better parameters. 

In [146]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000221C47A9090>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=MultinomialNB()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000221C47A9090>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=MultinomialNB()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf':

In [147]:
parameters = {

    'clf__estimator__alpha': [0.1, 0.5]
       
}

cv = GridSearchCV(pipeline,param_grid=parameters)

cv.fit(X_train,y_train)

c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokeni

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000221C47A9090>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=MultinomialNB()))]),
             param_grid={'clf__estimator__alpha': [0.1, 0.5, 1.0, 10.0]})

In [148]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__alpha': 0.1}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [175]:
pipeline_tuned = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=MultinomialNB(alpha=0.1)))
])

pipeline_tuned.fit(X_train , y_train)
y_pred_tuned = pipeline_tuned.predict(X_test)

for col in range(y_pred_tuned.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred_tuned[:,col],labels=np.unique(y_pred_tuned[:,col])))
    print(col)

c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

           0       0.67      0.36      0.47      1844
           1       0.82      0.95      0.88      5966
           2       0.63      0.22      0.32        55

    accuracy                           0.80      7865
   macro avg       0.71      0.51      0.56      7865
weighted avg       0.79      0.80      0.78      7865

0
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      6488
           1       0.74      0.55      0.63      1377

    accuracy                           0.89      7865
   macro avg       0.82      0.76      0.78      7865
weighted avg       0.88      0.89      0.88      7865

1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7838
           1       0.00      0.00      0.00        27

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [163]:
y_train


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
2935,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3395,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4927,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4898,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5237,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12076,1,1,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
15754,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
22896,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
pipeline_SVC = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=LogisticRegression()))
]) 

pipeline_SVC.fit(X_train , y_train)
y_pred_SVC = pipeline_SVC.predict(X_test)

for col in range(y_pred_SVC.shape[1]):

    print(classification_report(y_test.iloc[:,col],y_pred_SVC[:,col]))

c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\LocalAdmin\anaconda3\envs\Env_1\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.